In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

warnings.filterwarnings("ignore")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Model trained on corrected train.csv check out https://www.kaggle.com/nbroad/corrected-train-csv-feedback-prize detailed notebook on how to get the file.

* Loads datafrom dataframe to Datasets
* Uses the Longformer using with 1024 tokens
* Trainer API with fp16 enabled so as to optimize the training processes.
 

## Do Upvote if you find it usefull, It keeps me motivated to do more quality work, Thanks!


In [ ]:
train_df = pd.read_csv('/kaggle/input/corrected-train-csv-feedback-prize/corrected_train.csv')

train_df.columns

In [ ]:
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
# test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts.head()

In [ ]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))[:]):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
# train_texts['text'] = test_texts['text'].apply(lambda x:x.split())
train_text_df.head()

In [ ]:
train_df.head()

In [ ]:
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split().__len__()
#     entities = []
    entities = ["O" for i in range(total)]
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['new_predictionstring'].split()
        for li in list_ix[1:]:
#             print(li, entities)
            entities[int(li)] = f"I-{discourse}"
        entities[int(list_ix[0])] = f"B-{discourse}"
    all_entities.append(entities)

In [ ]:
# train_text_df = pd.read_csv("/kaggle/input/feedback-prize-ner-tagged-data/feedback_prize_ner_tagged_data.csv")
# train_text_df.head()

In [ ]:
# import ast
# train_text_df['entities'] = train_text_df['entities'].apply(lambda x:ast.literal_eval(x))

# print(train_text_df['entities'].values[0])

In [ ]:
train_text_df['entities'] = all_entities

train_text_df.head()

In [ ]:
# train_text_df[train_text_df.apply(lambda x: x['text'].split().__len__() == len(x['entities']), axis=1)]

In [ ]:
train_text_df = train_text_df[:]

### Define config and Load model, tokenizer

In [ ]:
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch import cuda
import torch

In [ ]:
config = {'model_name': '/kaggle/input/allenailongformerbase4096/longformer/',
         'max_length': 1024,
         'train_batch_size':4,
         'valid_batch_size':8,
         'epochs':5,
         'learning_rate':5e-05,
         'max_grad_norm':10,
          'warmup':0.1,
          "grad_acc":8,
          "model_save_path":"long-former",
         'device': 'cuda' if cuda.is_available() else 'cpu'}

output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [ ]:
train_text_df['labels'] = train_text_df['entities'].apply(lambda x: [labels_to_ids[i] for i in x])

In [ ]:
train_text_df.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model_name'], add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(config['model_name'],
                                                     num_labels=len(output_labels))

### Test to make sure is_split_into_words, return_offsets parameters are working properly

In [ ]:
converted = tokenizer(train_text_df.loc[0].values[1].split(),
                      is_split_into_words=True)

In [ ]:
print(converted, converted.word_ids())

In [ ]:
len(converted.word_ids()), len(train_text_df.loc[0].values[1].split())

In [ ]:
print(tokenizer.convert_ids_to_tokens(converted['input_ids'][:]), len(tokenizer.convert_ids_to_tokens(converted['input_ids'])))

### Tokenize the data

* Load data into huggingface datasets.
* Make sure you take care of sub-word tokenizing problem when adding labels to tokenized data.
* use .map to map tokenizer_data function to data
* Look at one sample to see whether mapping is done correctly or not


In [ ]:
def tokenizer_data(example):
    encoding = tokenizer(example['text'].split(),
                         is_split_into_words=True,
                         truncation=True,
                         padding='max_length',
                         max_length=config['max_length'])
#     i = 0
    labels = example['labels']
    encoded_labels = np.ones(len(encoding["input_ids"]), dtype=int) * -100
    previous_idx = None
    word_idx = encoding.word_ids()
    for i,idx in enumerate(word_idx):
        if idx != previous_idx and idx is not None:
#             print(idx)
            encoded_labels[i] = labels[idx]
#             i += 1
        previous_idx = idx
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    item['labels'] = torch.as_tensor(encoded_labels)
    return item

In [ ]:
dataset = datasets.Dataset.from_pandas(train_text_df)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

In [ ]:
text = dataset['train'][0]

# print(text['text'], text['entities'], text['labels'])

In [ ]:
print(text['text'].split())

In [ ]:
converted = tokenizer_data(text)

print(converted)

In [ ]:
converted

i=0
for token, label in zip(tokenizer.convert_ids_to_tokens(converted["input_ids"]), 
                        converted["labels"]):
#     print(token)
    if label != -100:
        print(token, '@@',text['text'].split()[i], ids_to_labels[label.item()],'@@', text['entities'][i])
        i+=1
    if i == 15:
        break

In [ ]:
dataset = dataset.map(tokenizer_data)

dataset

In [ ]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',
                                         'labels'])

dataset

### Check the shape of inputs_ids and attention_mask are same or not

In [ ]:
for a in dataset['train']:
    if a['input_ids'].shape[0] != a['attention_mask'].shape[0]:
        print(a)
        break

### Define training argument and train the model

In [ ]:
trainer_args = TrainingArguments('test_trainer',
                                report_to='none',
                                 num_train_epochs=config['epochs'],
                                evaluation_strategy ='epoch',
                                per_device_train_batch_size=config['train_batch_size'],
                                per_device_eval_batch_size=config['valid_batch_size'],
                                fp16=True,
                                save_strategy = "epoch",
                                 warmup_ratio= config['warmup'],
                                 gradient_accumulation_steps=config['grad_acc'],
                                 logging_strategy="epoch",
                                 save_total_limit=1
                                )

trainer = Trainer(model=model,
                  args=trainer_args, 
                  train_dataset = dataset['train'],
                  eval_dataset=dataset['test'],
#                   data_collator = data_collator,
                  tokenizer=tokenizer)

In [ ]:
trainer.train()

In [ ]:
device = config['device']

### Write inference function, loop through the test_text and dump into submission file

In [ ]:
trainer.model.eval()
def inference(sentence):
    inputs = tokenizer(sentence.split(),
                        is_split_into_words=True,
                        padding='max_length', 
                        truncation=True, 
                        max_length=4096,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = trainer.model(input_ids=ids, attention_mask=mask, return_dict=False)
#     print(outputs)
    logits = outputs[0]
    
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    print(logits.shape, active_logits.shape, flattened_predictions.shape)
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    previous_idx = None
    for idx, mapping in enumerate(inputs.word_ids()):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping is not None and mapping != previous_idx:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
#         else:
#             if idx == 1:
#                 prediction.append(wp_preds[idx][1])
#                 out_str.append(wp_preds[idx][0])
#             continue
        previous_idx = mapping
    return prediction, out_str

In [ ]:
te = test_texts.iloc[3]['text']

In [ ]:
len(inference(te)[0]), len(te.split())

In [ ]:
final_preds = []
import pdb
for i in tqdm(range(len(test_texts))):
    idx = test_texts.id.values[i]
    pred, _ = inference(test_texts.text.values[i])
#     print(idx.split(), pred)
    pred = [x.replace('B-','').replace('I-','') for x in pred]
    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
        if cls == 'O':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1
            
        if cls != 'O' and cls != '' and end - j > 7:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))
        
        j = end
        
# print(final_preds[1])

In [ ]:
len(final_preds)

In [ ]:
test_df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test_df

sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns

sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)